In [0]:
sentences_test = ["Add brace new line code style for switch expressions","New line setting for C# switch expression is missing"]
with tf.Session() as session:

  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  sentences_embed = session.run(embed(sentences_test))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(sentences_embed[0])
sts_encode2 = tf.nn.l2_normalize(sentences_embed[1])
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2))
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)

In [78]:
import numpy as np
np.inner(sentences_embed[0], sentences_embed[1])


0.8265699

In [77]:
from scipy.spatial import distance
test = distance.euclidean(sentences_embed[0], sentences_embed[1])
test

0.5889483690261841

In [80]:
import spacy

nlp = spacy.load('en')

#search_doc = nlp("Add brace new line code style for switch expressions")
#main_doc = nlp("New line setting for C# switch expression is missing")

search_doc = nlp("New line setting for C# switch expression is missing")
main_doc = nlp("Add brace new line code style for switch expressions")
print(main_doc.similarity(search_doc))

0.7846811147040956


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
